In [9]:
# Importing the libraries
import pandas as pd
import kagglehub
import os

In [10]:
path = kagglehub.dataset_download("chavindudulaj/vehicle-maintenance-data")
print(path)
# Load the dataset
csv_path = os.path.join(path, "vehicle_maintenance_data.csv")
data = pd.read_csv(csv_path)
print(data.head())

/root/.cache/kagglehub/datasets/chavindudulaj/vehicle-maintenance-data/versions/1
  Vehicle_Model  Mileage Maintenance_History  Reported_Issues  Vehicle_Age  \
0         Truck    58765                Good                0            4   
1           Van    60353             Average                1            7   
2           Bus    68072                Poor                0            2   
3           Bus    60849             Average                4            5   
4           Bus    45742                Poor                5            1   

  Fuel_Type Transmission_Type  Engine_Size  Odometer_Reading  \
0  Electric         Automatic         2000             28524   
1  Electric         Automatic         2500            133630   
2  Electric         Automatic         1500             34022   
3    Petrol         Automatic         2500             81636   
4    Petrol            Manual         2000             97162   

  Last_Service_Date Warranty_Expiry_Date Owner_Type  Insurance_P